In [186]:
%reload_ext autoreload
%autoreload 2
import plotly.express as px
import sys
from dotenv import load_dotenv
import os
load_dotenv()

# Access the environment variables
ROOT = os.getenv("ROOT")
sys.path.append(ROOT)
from src.data_loader import load_reviews
from src.processing import reviews_processing
from src.nlp.sentiment_analysis import apply_reviews_sentiment
from src.recommendation import recommendation_system

In [187]:
df = load_reviews(category="All_beauty", frac=0.01)

In [188]:
df = reviews_processing(df=df, clean_text=False)

In [189]:
df = apply_reviews_sentiment(df=df)

In [190]:
px.histogram(df, x='sentiment', nbins=50, title="Sentiment scores histogram").show()

In [191]:
px.violin(df, x='rating', y='sentiment', title="Violin plots for sentiment by rating", box=True).show()

In [192]:
product_id = df["asin"].values[0]

recommendations = recommendation_system(
    product_id=product_id,
    df=df,
    method="tf-idf",
    top_n=10,
)

In [193]:
recommendations

,rating,review_title,review,images_x,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,description,price,images_y,videos,store,categories,details,bought_together,review_input,sentiment
4145,3,This polish doesn't dry without a polish dryer,"Great color but polish doesn't air dry, it nee...",[],B08RYR533M,B08RYR533M,AFP6G2ZY3SXAL3SIZYBMZ7PLKTFA,2021-06-14 01:42:55.416,0,True,...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],GERSHION,[],"{'Brand': 'GERSHION', 'Item Form': 'Gel', 'Col...",NaN,This polish doesn't dry without a polish dryer...,-0.066667
6325,5,"Nice polish, great color.",First time trying an antifungal polish and it'...,[],B00B18O19G,B0BNWSR278,AFKYXQRBPQS5V3KJ755KFILNFXQA,2022-01-26 00:24:52.979,0,True,...,[Available in a kaleidoscope of over 40 vibran...,18.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Dr.'s Remedy featured on The Racha...,Dr.'s Remedy,[],"{'Brand': 'Dr.'s Remedy', 'Item Form': 'Liquid...",NaN,"Nice polish, great color.First time trying an ...",0.490000
1582,5,Great product!!,This gel polish remover really works well. I’v...,[{'small_image_url': 'https://m.media-amazon.c...,B09GYQ81R5,B09GYQ81R5,AFX6RRF3GQHURISWFHBHTCNAXFDA,2022-08-04 12:39:35.033,0,True,...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Gel nail polish remover', 'url': '...",AIBRIT,[],"{'Brand': 'AIBRIT', 'Item Form': 'Liquid', 'It...",NaN,Great product!!This gel polish remover really ...,0.473333
4075,3,Gel nail polish remover,This gel nail polish remover does remove nail ...,[{'small_image_url': 'https://m.media-amazon.c...,B0BR96M4Y8,B0BR96M4Y8,AHQE7ZPD7VXBJ5TCCMHT53PF7SWA,2023-02-12 23:37:55.925,0,False,...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Gel Nail Polish Remover, quick and...",ZODENIS,[],{'Package Dimensions': '8.11 x 3.7 x 0.98 inch...,NaN,Gel nail polish removerThis gel nail polish re...,0.277778
6000,5,Highly recommend,Best gel nail polish ever,[],B072R4GHNL,B072R4GHNL,AEBMUK7RROJM7DNBPDX45EGTMX6Q,2018-08-06 18:43:25.983,0,True,...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Gellen,[],"{'Brand': 'Gellen', 'Item Form': 'Gel', 'Type'...",NaN,Highly recommendBest gel nail polish ever,0.160000
2276,5,Nail Polish Remover,I don’t like the smell of acetone. I bought th...,[{'small_image_url': 'https://images-na.ssl-im...,B01AYTGWA8,B01AYTGWA8,AEAZYANKUKSNXGDWBPIQL4KOXXPA,2016-04-06 06:09:53.000,0,False,...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Absolutely No Chemical Smell, Work...",Pei Mei,[],"{'Brand': 'Pei Mei', 'Item Form': 'Oil', 'Item...",NaN,Nail Polish RemoverI don’t like the smell of a...,0.130392
1192,5,Removing gel polish has never been so easy,This product is amazing. It makes an easy job...,[],B09GYQ81R5,B09GYQ81R5,AH3NFR56OFKXPDJLZSFG7FUWAUYQ,2022-06-09 02:51:05.326,3,True,...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Gel nail polish remover', 'url': '...",AIBRIT,[],"{'Brand': 'AIBRIT', 'Item Form': 'Liquid', 'It...",NaN,Removing gel polish has never been so easyThis...,0.433333
6319,5,Everything needed to remove gel polish.,This review is for the VANALIA Gel Nail Polish...,[{'small_image_url': 'https://m.media-amazon.c...,B09Y1Z581L,B09Y1Z581L,AESQ3D5NENZDKGZGS5RNBHE4T43A,2022-08-05 02:56:03.124,0,False,...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'How to use vanalia gel nail polish...,VANALIA,[],{'Package Dimensions': '4.9 x 4.1 x 1.3 inches...,NaN,Everything needed to remove gel polish.This re...,0.175000
592,5,Powerful moisturizing polish remover!,Kwik off removed my nail polish faster than an...,[{'small_image_url': 'https://m.media-amazon.c...,B01IAEHV4U,B01IAEHV4U,AFVL4D4DGXUSZTZTLE5FLKEIWO7Q,2022-01-13 11:09:26.513,0,False,...,[],21.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Artificial nail polish remover wit...,Sally Hansen,[],"{'Brand': 'Sally Hansen', 'Item Form': 'Liquid